In [2]:
from lib import models_siamese, graph, abide_utils
import numpy as np
import os
import time
import scipy

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [3]:
rs = 665

print("Random state is %d" % rs)
prng = np.random.RandomState(rs)

Random state is 665


In [4]:
# get the subjects feature matrix
# import h5py
subset = range(500)
all_mats = scipy.io.loadmat('/ysm-gpfs/home/sg972/project/data/all_mats_rest.mat')
all_mats = all_mats['all_mats']
all_mats = np.array(all_mats)
all_mats = np.transpose(all_mats[:, :, subset], (2,0,1)) # get the rest data
print(np.shape(all_mats))

(500, 268, 268)


In [5]:
# get the subjects label
num_train = np.shape(all_mats)[0]
label = scipy.io.loadmat('/ysm-gpfs/home/sg972/project/data/male_index.mat')
label = label['male_index']
all_labels = np.zeros((515, 1))
all_labels[label] = 1
all_labels = all_labels[subset]
print(np.shape(all_labels))

(500, 1)


In [6]:
def split_data(site, train_perc):
    """ Split data into training and test indices """
    train_indices = []
    test_indices = []

    for s in np.unique(site):
        # Make sure each site is represented in both training and test sets
        id_in_site = np.argwhere(site == s)[:,0]
        num_nodes = len(id_in_site)
        train_num = int(train_perc * num_nodes)
        prng.shuffle(id_in_site)
        train_indices.extend(id_in_site[:train_num])
        test_indices.extend(id_in_site[train_num:])

    # print("Number of labeled samples %d" % len(train_indices))

    return train_indices, test_indices


def prepare_pairs(X, y, indices):
    """ Prepare the graph pairs before feeding them to the network """
    N, M, F = X.shape
    n_pairs = int(len(indices) * (len(indices) - 1) / 2)
    triu_pairs = np.triu_indices(len(indices), 1)
    
    site_pairs = np.ones(int(n_pairs))
    
    X_pairs = np.ones((n_pairs, M, F, 2))
    X_pairs[:, :, :, 0] = X[indices][triu_pairs[0]]
    X_pairs[:, :, :, 1] = X[indices][triu_pairs[1]]

#     site_pairs = np.ones(int(n_pairs))
#     print(site[indices][triu_pairs[0]] != site[indices][triu_pairs[1]])
#     site_pairs[site[indices][triu_pairs[0]] != site[indices][triu_pairs[1]]] = 0

    y_pairs = np.ones(int(n_pairs))
    y_index = y[triu_pairs[0]] != y[triu_pairs[1]]
    y_pairs[y_index.flatten()] = 0  # -1

    print(n_pairs)

    return X_pairs, y_pairs, site_pairs

In [7]:
# training_num = int(num_train*0.8)
training_num = int(num_train*0.13)*6
site = np.ones(num_train)
tr_idx, test_idx = split_data(site, 0.7)
prng.shuffle(test_idx)
subs_to_add = training_num - len(tr_idx)  # subjects that need to be moved from testing to training set
tr_idx.extend(test_idx[:subs_to_add])
test_idx = test_idx[subs_to_add:]
print("The num of training samples are: ")
print(len(tr_idx))


The num of training samples are: 
390


In [8]:
# Compute all the combo pairs
all_combs = []
tr_mat = np.array(tr_idx).reshape([int(training_num / 6), 6])
for i in range(3):
    x1 = tr_mat[:, i * 2].flatten()
    x2 = tr_mat[:, i * 2 + 1].flatten()
    combs = np.transpose([np.tile(x1, len(x2)), np.repeat(x2, len(x1))])
    all_combs.append(combs)

all_combs = np.vstack(all_combs)

print(all_combs.shape) #(training_num/6)*(training_num/6)*3

(12675, 2)


In [9]:
n, m, f = all_mats.shape
X_train = np.ones((all_combs.shape[0], m, f, 2), dtype=np.float32)
y_train = np.ones(all_combs.shape[0], dtype=np.int32)

for i in range(all_combs.shape[0]):
    X_train[i, :, :, 0] = all_mats[all_combs[i, 0], :, :]
    X_train[i, :, :, 1] = all_mats[all_combs[i, 1], :, :]
    if all_labels[all_combs[i, 0]] != all_labels[all_combs[i, 1]]:
        y_train[i] = 0  # -1
        
print(np.shape(X_train))

(12675, 268, 268, 2)


In [10]:
# Get the graph structure

coords_file = '/ysm-gpfs/home/sg972/project/code/gcn_metric_learning-master/shen_coords.csv'
coords = np.loadtxt(coords_file, delimiter=',')

dist, idx = graph.distance_scipy_spatial(coords, k=10, metric='euclidean')
A = graph.adjacency(dist, idx).astype(np.float32)

graphs = []
for i in range(3):
    graphs.append(A)

# Calculate Laplacians
L = [graph.laplacian(A, normalized=True) for A in graphs]
print(np.shape(L[0]))

(268, 268)


In [11]:
# # print(type(A))
# temp = np.abs(np.squeeze(all_mats[1,:,:]))
# graphs = []
# for i in range(3):
#     graphs.append(scipy.sparse.csr.csr_matrix(temp).astype(np.float32))
# L = [graph.laplacian(B, normalized=True) for B in graphs]

In [12]:
# all_mats = []
X_test, y_test, site_test = prepare_pairs(all_mats, all_labels, test_idx)

5995


In [13]:
n, m, f, _ = X_train.shape

In [26]:
params = dict()

In [36]:
# Graph Conv-net
features = 20
K = 3
params['num_epochs']     = 50
params['batch_size']     = 10
params['eval_frequency'] = X_train.shape[0] / (params['batch_size'] * 2)
# params['eval_frequency'] = 50

In [37]:
# Building blocks.
params['filter']         = 'chebyshev5'
params['brelu']          = 'b2relu'
params['pool']           = 'apool1'

In [38]:
# Architecture.
params['F']              = [features, features]   # Number of graph convolutional filters.
params['K']              = [K, K]   # Polynomial orders.
params['p']              = [1, 1]     # Pooling sizes.
params['M']              = [1]    # Output dimensionality of fully connected layers.
params['input_features'] = f
params['lamda']          = 0.35
params['mu']             = 0.6

In [30]:
# Optimization.
params['regularization'] = 5e-3
params['dropout']        = 0.8 #0.5
params['learning_rate']  = 1e-3
params['decay_rate']     = 0.95
params['momentum']       = 0
params['decay_steps']    = X_train.shape[0] / params['batch_size']

params['dir_name']       = 'siamese_' + time.strftime("%Y_%m_%d_%H_%M") + '_feat' + str(params['F'][0]) + '_' + \
                           str(params['F'][1]) + '_K' + str(K) + '_state' + str(rs)

In [34]:
# Save logs to folder
path = os.path.dirname(os.path.abspath('__file__'))
log_path = os.path.join(path, 'logs', params['dir_name'])
os.makedirs(log_path)
# print(log_path)
print(params)

FileExistsError: [Errno 17] File exists: '/ysm-gpfs/project/sg972/code/gcn_metric_learning-master/logs/siamese_2017_11_23_01_46_feat1_1_K3_state665'

In [ ]:
# Run model
site_train = np.ones(all_combs.shape[0], dtype=np.int32)

model = models_siamese.siamese_cgcnn_cor(L, **params)
accuracy, loss, t_step, scores_summary = model.fit(X_train, y_train, site_train, X_test, y_test, site_test)
print('Time per step: {:.2f} ms'.format(t_step*1000))

NN architecture
  input: M_0 = 268
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 268 * 20 / 1 = 5360
    weights: F_0 * F_1 * K_1 = 268 * 20 * 3 = 16080
    biases: M_1 * F_1 = 268 * 20 = 5360
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 268 * 20 / 1 = 5360
    weights: F_1 * F_2 * K_2 = 20 * 20 * 3 = 1200
    biases: M_2 * F_2 = 268 * 20 = 5360
  layer 3: logits (softmax)
    representation: M_3 = 1
    weights: M_2 * M_3 = 5360 * 1 = 5360
    biases: M_3 = 1
(10, 268, 268, 2)
step 2535 / 63375 (epoch 2.00 / 50):
  learning_rate = 9.50e-04, loss_average = nan
  training samples: 12675, AUC : 0.55, loss: 2.1180e-01
  validation samples: 5995, AUC : 0.51, loss: 2.1205e-01
  time: 44s (wall 41s)
step 5070 / 63375 (epoch 4.00 / 50):
  learning_rate = 8.57e-04, loss_average = nan
  training samples: 12675, AUC : 0.55, loss: 2.1024e-01
  validation samples: 5995, AUC : 0.51, loss: 2.1042e-01
  time: 88s (wall 82s)
step 7605 / 63375 (epoch 6.00 / 50):
  learning_rate =

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

device_lib.list_local_devices()